In [1]:
import pandas as pd
import numpy as np
import os
import io
import sqlite3
from bs4 import BeautifulSoup

In [2]:
def extract_features(bs_content, contextRef, fund_id):
    
    #General Description
    general = bs_content.find_all('CategoriaFIM')[0].text.split('\n')
    fund_type = general[1]
    fund_focus = general[2]
    fund_risk = general[3]    
    
    #AUM
    all_aum = bs_content.find_all('Patrimonio', {'contextRef': contextRef + '_ia'})
    aum = sum([float(i.text) for i in all_aum])
    
    #Shareholders
    all_sh = bs_content.find_all('NumeroParticipes', {'contextRef': contextRef + '_ia'})
    sh = sum([float(i.text) for i in all_sh])    
    
    #Fees
    all_fees = bs_content.find_all('ComisionGestionCobrada', {'contextRef': contextRef + '_daa'})
    fees = sum([float(i.text) for i in all_fees])
    
    #Portfolio Rotation
    port_rot = float(bs_content.find_all('IndiceRotacionCartera', {'contextRef': contextRef + '_da'})[0].text)
    
    #Investment Policy
    inv_pol = bs_content.find_all('PoliticaInversion')[0].text.replace('\n', '').encode('latin-1').decode('utf-8')
    
    df_features = pd.DataFrame(data={'Fund ID': fund_id, 'Fund Type': fund_type, 'Fund Focus': fund_focus, 'Fund Risk': fund_risk,
                                     'AUM': aum, 'Fees': fees, 'Rotation': port_rot, 'Inv Policy': inv_pol}, index=[0])
    
    return df_features

In [3]:
list_files = os.listdir('Info Pública')
list_files.remove('Funds DB')

all_prts = pd.DataFrame(columns=['Fund ID', 'Fund Type', 'Fund Focus', 'Fund Risk', 'AUM', 'Fees', 'Rotation', 'Inv Policy'])

for i_file in list_files:
    
    try:
        #Convert xml format to Beautifulsoup
        content = []
        # Read the XML file        
        with open('Info Pública/' + i_file, 'rb') as file:
            i_bs_content = BeautifulSoup(file.read(), 'xml')        

        #contextRef is an attribute of each file
        #adding '_ia' makes reference to current value and '_ipp' makes reference to past value   
        i_contextRef = i_bs_content.find('context')['id'].replace('_da','')    

        i_port = extract_features(i_bs_content, i_contextRef, i_file.replace('.xml',''))

        all_prts = pd.concat([all_prts, i_port], ignore_index=True)
    
    except:
        continue
    
all_prts.to_pickle('portfolios_features.pickle')    
all_prts.head(10)

,Fund ID,Fund Type,Fund Focus,Fund Risk,AUM,Fees,Rotation,Inv Policy
0,103-1081,08,07,Alto,5.265588e+07,1.50,0.70,FI RENTA VARIABLE MIXTA INTERNACIONALSe invert...
1,103-1230,08,09,Alto,1.398579e+09,1.75,0.71,FI RENTA VARIABLE INTERNACIONALEl objetivo de ...
2,103-2269,08,04,4 en una escala del 1 al 7,1.278797e+08,1.00,0.29,FI RENTA FIJA MIXTA EUROSe invertirï¿½ hasta e...
3,103-377,08,09,Alto,1.628275e+09,1.75,0.76,"FI, RENTA VARIABLE INTERNACIONAL.El objetivo d..."
4,103-4290,08,09,Alto,3.945306e+08,1.75,0.82,FI RENTA VARIABLE INTERNACIONALEl objetivo de ...
5,103-4422,08,09,Alto,1.002486e+08,1.50,0.39,FI RENTA VARIABLE INTERNACIONALAl menos el 75%...
6,103-502,08,08,Alto,1.580924e+08,1.75,1.18,"FI, RENTA VARIABLE EUROLas inversiones del fon..."
7,103-5172,08,09,6,1.232087e+07,2.55,1.78,La gestiï¿½n toma como referencia la rentabili...
8,103-5279,08,20,1,1.756381e+08,0.15,0.00,RENTA FIJA EUROEl fondo invierte el 100% de la...
9,103-5330,08,09,7,2.366307e+07,1.75,0.31,Al menos el 75% de la exposiciï¿½n total se in...


In [5]:
con = sqlite3.connect('Portfolios_Features.db')
cur = con.cursor()
#cur.execute('DROP TABLE IF EXISTS Portfolios_Features')
cur.execute('CREATE TABLE Portfolios_Features (Fund_ID TEXT, Fund_Type TEXT, Fund_Focus TEXT, Fund_Risk TEXT, AUM REAL, Fees REAL, Rotation REAL, Inv_Policy TEXT)')

records = list(all_prts.to_records(index=False))

cur.executemany('INSERT INTO Portfolios_Features VALUES (?, ?, ?, ?, ?, ?, ?, ?)', records)

con.commit()

con.close()